# Exercício 1 
#### A partir da base de dados propaganda.csv faça três modelos de regressão simples para comparar as propagandas de TV, rádio e jornal. É imprescindível realizar:

1 – Análise descritiva

2 – Análise de correlação

3 – Modelo de regressão

4 – Analise de resíduos

5 – Análise do Modelo

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import lilliefors
import scipy.stats as stats



### 1 – Análise descritiva

Carregando a base de dados

In [ ]:
dados = pd.read_csv('propaganda.csv')

Analise descritiva das medidas de tendencia central via tabela

In [ ]:
dados.describe()

Verificação do tamanho da base de dados

In [ ]:
dados.shape

Visualização das 5 primeiras linhas 

In [ ]:
dados.head()

Verificação do tipo das variáveis

In [ ]:
dados.dtypes

Verificação de valores ausentes e nulos

In [ ]:
dados.isnull().sum()

A base de dados não possui dados faltantes ou nulos e todos os dados são do mesmo tipo.

In [ ]:
# Análise gráfica - Pairplot
sns.pairplot(dados, corner=True) # corner corta metade do gráfico pois é simetrico.
plt.suptitle("Análise Gráfica dos Dados de Propagandas")
plt.show()

Analise grafica para verificar a disposição dos dados e relação entre as variáveis

### 2 – Calculo de correlação


In [ ]:
# Calcula a matriz de correlação
correlation_matrix = dados.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="crest", fmt=".2f")
plt.title("Matriz de Correlação")
plt.show()


Podemos ver claramente que a correlacão de Vendas com propagandas em TV é maior que na Radio que é maior que no Jornal

### 3 – Faça o modelo de regressão

In [ ]:
# Definindo a variável dependente (Vendas) e independente (TV)
X = dados['TV']  # Variável independente (TV)
X_radio = dados['Radio']
X_jornal = dados['Jornal']
y = dados['Vendas']  # Variável dependente (Vendas)

# Adicionando uma constante à variável independente
X = sm.add_constant(X)
X_radio = sm.add_constant(X_radio)
X_jornal = sm.add_constant(X_jornal)

# Ajustando o modelo de regressão linear
modelo_tv = sm.OLS(y, X).fit()
modelo_radio = sm.OLS(y, X_radio).fit()
modelo_jornal = sm.OLS(y, X_jornal).fit()

# Sumário do modelo
modelo_tv_summary = modelo_tv.summary()
modelo_radio_summary = modelo_radio.summary()
modelo_jornal_summary = modelo_jornal.summary()


# sm.OLS() é usado para ajustar o modelo de regressão linear ordinária.
# sm.add_constant() é necessário pois o OLS não adiciona automaticamente a constante (intercepto).


### 4 – realize a analise de resíduos


In [ ]:
# Resíduos dos modelos
residuos_tv = modelo_tv.resid
residuos_radio = modelo_radio.resid
residuos_jornal = modelo_jornal.resid

# Gráfico de homocedasticidade
plt.figure(figsize=(10, 6))

plt.subplot(1, 3, 1)
sns.scatterplot(x=modelo_tv.fittedvalues, y=residuos_tv)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Homocedasticidade - TV')

plt.subplot(1, 3, 2)
sns.scatterplot(x=modelo_radio.fittedvalues, y=residuos_radio)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Homocedasticidade - Rádio')

plt.subplot(1, 3, 3)
sns.scatterplot(x=modelo_jornal.fittedvalues, y=residuos_jornal)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Homocedasticidade - Jornal')

plt.tight_layout()
plt.show()

No grafico 3, observa-se que os pontos estão aleatoriamente espalhados em torno da linha zero, com uma faixa constante de dispersão, isso pode inficar homocedasticidade. 
Já nos gráficos 1 e 2, existe um padrão claro e uma ampla dispersão, podendo indicar heterocedasticidade.

#### Teste de Homocedasticidade com Breusch-Pagan

In [ ]:
# Calcula os resíduos do modelo
# Criação de uma lista de tuplas com os modelos e seus respectivos nomes
modelos = [
    ("modelo_tv", modelo_tv, dados['TV']),
    ("modelo_radio", modelo_radio, dados['Radio']),
    ("modelo_jornal", modelo_jornal, dados['Jornal'])
]

for nome, modelo,dados in modelos:
    estatistica, p, f, fp = het_breuschpagan(modelo.resid, modelo.model.exog)
    print(f'Estatísticas de teste para {nome}: {format(estatistica)}')
    print(f'p-valor: {p}')
    print(f'f-valor: {fp}')
    print(f'f_p-valor: {f}')


#### Breusch-Pagan:
Se o p-valor do teste menor que 0.05, indica a presença de heterocedasticidade e pode ser necessário ajustar o modelo ou usar métodos robustos para estimativa de variância.
modelo_tv     p-valor < 0.05
modelo_radio  p-valor < 0.05
modelo_jornal p-valor > 0.05

#### Teste de Normalidade dos Resíduos com Shapiro-Wilk

In [ ]:
for nome, modelo,dados in modelos:
    estatistica_shapiro,p_shapiro = stats.shapiro(modelo.resid)
    print(f'Estatísticas de teste {nome}: {format(estatistica_shapiro)}')
    print('p-valor: {}'.format(p_shapiro))

#### Shapiro:
Se p-valor do shapiro é superior a 0.05,isso indica que os resíduos seguem aproximadamente uma distribuição normal.

- modelo_tv     p-valor > 0.05
- modelo_radio  p-valor > 0.05
- modelo_jornal p-valor < 0.05

#### Análise de normalidade QQ-plot

In [ ]:
 for nome, modelo, dados in modelos:
    stats.probplot(modelo.resid,dist="norm", plot=plt)
    plt.title(f"Normal Q-Q plot - {nome}")
    plt.show()

Aparentemente todos se aproximam da reta e parecem ser normais.

In [ ]:
 for nome, modelo, dados in modelos:
    residuos_padronizados = modelo.get_influence().resid_studentized_internal
    
    # Cria um gráfico de resíduos
    plt.figure(figsize=(8, 6))
    plt.scatter(dados, residuos_padronizados)
    plt.axhline(y=3, color='r', linestyle='--')
    plt.axhline(y=-3, color='r', linestyle='--')
    plt.title(f'Gráfico de Resíduos Padronizados para {nome}')
    plt.xlabel('Investimento em ' + nome.split("_")[1].capitalize())
    plt.ylabel('Resíduos Padronizados')
    
    # Mostrar o gráfico
    plt.show()

Nenhum dos residuos apresenta um outlier além de 3 e -3

#### Análise geral dos residuos: 
| Modelo  | Breusch-Pagan (Homocedasticidade) | Shapiro-Wilk (Normalidade) | QQ-Plot | Outliers |
|---------|----------------------------------|----------------------------|---------|----------|
| TV      | ❌                                | ✅                          | ✅       | ✅        |
| Rádio   | ❌                                | ❌                          | ✅       | ✅        |
| Jornal  | ✅                                | ❌                          | ✅       | ✅        |

### 5 – Análise do Modelo

In [ ]:
for nome, modelo, dados in modelos:
    print(str(modelo.summary()))

| Modelo  | Teste T (Significância dos Coeficientes) | Teste F (Significância Global) | R² Ajustado |
|---------|-----------------------------------------|--------------------------------|-------------|
| TV      | ✅                                        | ✅                              | 0.610       |
| Rádio   | ✅                                        | ✅                              | 0.329       |
| Jornal  | ✅                                        | ✅                              | 0.047       |

#### Teste T:
O Teste T avalia a significância estatística de cada coeficiente no modelo. Um p-valor baixo (< 0.05) indica que o coeficiente é significativo. Nos resultados fornecidos, todos os modelos mostraram coeficientes significativos (p-valor < 0.05 para os coeficientes de TV, Rádio e Jornal).

#### Teste F:
O Teste F avalia a significância global do modelo. Um p-valor baixo (< 0.05) sugere que o modelo, como um todo, é significativo. Todos os modelos (TV, Rádio e Jornal) apresentaram p-valores baixos no Teste F, indicando que são estatisticamente significativos.

#### R² Ajustado:
O R² Ajustado reflete a porcentagem da variabilidade da variável dependente explicada pelo modelo, ajustada pelo número de preditores. Valores mais altos indicam um melhor ajuste. No caso, o modelo de TV tem um R² Ajustado de 0.610, indicando que cerca de 61% da variabilidade em Vendas é explicada pelo modelo. Os modelos de Rádio e Jornal têm R² Ajustado mais baixos, indicando um ajuste menos eficaz.

### Grafico dos medelos ajustados

In [ ]:
fig = sm.graphics.plot_ccpr(modelo_tv, "TV")
fig.tight_layout(pad=1.0)

In [ ]:
fig = sm.graphics.plot_ccpr(modelo_radio, "Radio")
fig.tight_layout(pad=1.0)

In [ ]:
fig = sm.graphics.plot_ccpr(modelo_jornal, "Jornal")
fig.tight_layout(pad=1.0)